In [1]:
# some magic so that the notebook will reload external python modules;
# see https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [1]:
import json
import random
import os
import hydra
import logging

import torch
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
# from pytorch_lightning.plugins import DDPPlugin
from avg_ckpts import ensemble
from datamodule.data_module import DataModule
from datamodule.grid_dataset import GridDataset

In [3]:
from datamodule.transforms import VideoTransform

In [10]:
from datamodule.grid_dataset import GridDataset
root_dir = "/ssd_scratch/cvit/vanshg/preprocessed_grid/video"
label_path = "/ssd_scratch/cvit/vanshg/preprocessed_grid/labels/s1_label.csv"
data_size = 0.8

dataset = GridDataset(root_dir, label_path, VideoTransform("train"), data_size=data_size)

Size of self.list: 800


In [12]:
for data in dataset:
    print(type(data))
    print(data['input'].shape)
    print(data['target'])
    break

<class 'dict'>
torch.Size([75, 1, 88, 88])
tensor([ 941,  969,  811, 2033, 4742, 3291])


In [6]:
from datamodule.phrase_dataset import PhraseDataset
root_dir = "/ssd_scratch/cvit/vanshg/vansh_phrases/preprocessed_phrases"
label_path = "/ssd_scratch/cvit/vanshg/vansh_phrases/phrases.json"
data_size = 0.8

dataset = PhraseDataset(root_dir, label_path, VideoTransform('train'), data_size=data_size)

Label Path : /ssd_scratch/cvit/vanshg/vansh_phrases/phrases.json, exists = True
Size of self.video_list: 80


In [9]:
label_path = "/ssd_scratch/cvit/vanshg/vansh_phrases/phrases.json"
dst_train_label_path = "/ssd_scratch/cvit/vanshg/vansh_phrases/train_phrases_60.json"
dst_test_label_path = "/ssd_scratch/cvit/vanshg/vansh_phrases/test_phrases_40.json"

f = open(label_path, 'r')

video_list = json.load(f)
rng = random.Random(69)
rng.shuffle(video_list)

train_video_list = video_list[:60]
test_video_list = video_list[60:]
print(len(train_video_list))
print(len(test_video_list))

with open(dst_train_label_path, 'w') as file:
    json.dump(train_video_list, file)

with open(dst_test_label_path, 'w') as file:
    json.dump(test_video_list, file)

60
40
